In [32]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb # imdb dataset is available in tensorflow library
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,SimpleRNN,Dense


In [33]:
##LOAD the imdb dataset
voc_size=10000 ## Vocabulary size
(x_train,y_train),(x_test,y_test) = imdb.load_data(num_words=voc_size)
print(f"Train data size:{x_train.shape},Train label size:{y_train.shape}")
print(f"Test data size:{x_train.shape},Test label size:{y_train.shape}")

Train data size:(25000,),Train label size:(25000,)
Test data size:(25000,),Test label size:(25000,)


In [34]:
x_train[0],y_train[0] ## it gives index for word at which it occures.i.e, it is ohe representation

([1,
  14,
  22,
  16,
  43,
  530,
  973,
  1622,
  1385,
  65,
  458,
  4468,
  66,
  3941,
  4,
  173,
  36,
  256,
  5,
  25,
  100,
  43,
  838,
  112,
  50,
  670,
  2,
  9,
  35,
  480,
  284,
  5,
  150,
  4,
  172,
  112,
  167,
  2,
  336,
  385,
  39,
  4,
  172,
  4536,
  1111,
  17,
  546,
  38,
  13,
  447,
  4,
  192,
  50,
  16,
  6,
  147,
  2025,
  19,
  14,
  22,
  4,
  1920,
  4613,
  469,
  4,
  22,
  71,
  87,
  12,
  16,
  43,
  530,
  38,
  76,
  15,
  13,
  1247,
  4,
  22,
  17,
  515,
  17,
  12,
  16,
  626,
  18,
  2,
  5,
  62,
  386,
  12,
  8,
  316,
  8,
  106,
  5,
  4,
  2223,
  5244,
  16,
  480,
  66,
  3785,
  33,
  4,
  130,
  12,
  16,
  38,
  619,
  5,
  25,
  124,
  51,
  36,
  135,
  48,
  25,
  1415,
  33,
  6,
  22,
  12,
  215,
  28,
  77,
  52,
  5,
  14,
  407,
  16,
  82,
  2,
  8,
  4,
  107,
  117,
  5952,
  15,
  256,
  4,
  2,
  7,
  3766,
  5,
  723,
  36,
  71,
  43,
  530,
  476,
  26,
  400,
  317,
  46,
  7,
  4,
  2,
  1029,
  

In [35]:
## Inspect sample review
sample_review = x_train[0]
sample_label = y_train[0]
print(f"sample review : {sample_review}")
print(f"label : {sample_label}")

sample review : [1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]
label : 1


In [36]:
## Mapping of word index to words (just for understanding)
word_index=imdb.get_word_index()
# word_index
reverse_word_index={value:key for key,value in word_index.items()}
reverse_word_index

{34701: 'fawn',
 52006: 'tsukino',
 52007: 'nunnery',
 16816: 'sonja',
 63951: 'vani',
 1408: 'woods',
 16115: 'spiders',
 2345: 'hanging',
 2289: 'woody',
 52008: 'trawling',
 52009: "hold's",
 11307: 'comically',
 40830: 'localized',
 30568: 'disobeying',
 52010: "'royale",
 40831: "harpo's",
 52011: 'canet',
 19313: 'aileen',
 52012: 'acurately',
 52013: "diplomat's",
 25242: 'rickman',
 6746: 'arranged',
 52014: 'rumbustious',
 52015: 'familiarness',
 52016: "spider'",
 68804: 'hahahah',
 52017: "wood'",
 40833: 'transvestism',
 34702: "hangin'",
 2338: 'bringing',
 40834: 'seamier',
 34703: 'wooded',
 52018: 'bravora',
 16817: 'grueling',
 1636: 'wooden',
 16818: 'wednesday',
 52019: "'prix",
 34704: 'altagracia',
 52020: 'circuitry',
 11585: 'crotch',
 57766: 'busybody',
 52021: "tart'n'tangy",
 14129: 'burgade',
 52023: 'thrace',
 11038: "tom's",
 52025: 'snuggles',
 29114: 'francesco',
 52027: 'complainers',
 52125: 'templarios',
 40835: '272',
 52028: '273',
 52130: 'zaniacs',

In [37]:
decoded_review =' '.join([reverse_word_index.get(i-3,'?') for i in sample_review])
"""Keras IMDB shifts all real word indices by +3 to leave room for special tokens.
When decoding, you subtract 3 (i-3) to map back to the original word index"""
decoded_review

"? this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert ? is an amazing actor and now the same being director ? father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for ? and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also ? to the two little boy's that played the ? of norman and paul they were just brilliant children are often left out of the ? list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you th

In [38]:
max_len=500
x_train=sequence.pad_sequences(x_train,maxlen=max_len)
x_test=sequence.pad_sequences(x_test,maxlen=max_len)

In [39]:
x_train

array([[   0,    0,    0, ...,   19,  178,   32],
       [   0,    0,    0, ...,   16,  145,   95],
       [   0,    0,    0, ...,    7,  129,  113],
       ...,
       [   0,    0,    0, ...,    4, 3586,    2],
       [   0,    0,    0, ...,   12,    9,   23],
       [   0,    0,    0, ...,  204,  131,    9]], dtype=int32)

In [40]:
x_train[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [41]:
## Making Simple RNN :-
voc_size=10000
max_len=500
model = Sequential()
model.add(Embedding(voc_size,128,input_length=max_len))
model.build((None,max_len))  # 👈 manually build
"""  In recent TensorFlow/Keras versions (2.13+), Sequential models do lazy building.
If you don’t provide an explicit input_shape (or you don’t call the model once with data), the model remains unbuilt."""
## Embedding layer that converts words in to vectors in specific demnsions
model.add(SimpleRNN(128,activation='relu'))

model.add(Dense(1,activation='sigmoid')) 
# sigmoid for binary output and for multiple output use softmax


In [42]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (None, 500, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_1 (SimpleRNN)        │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,313,025 (5.01 MB)

 Trainable params: 1,313,025 (5.01 MB)

 Non-trainable params: 0 (0.00 B)

In [43]:
model.compile(optimizer='adam',loss="binary_crossentropy",metrics=['accuracy'])

In [44]:
## Create an instance of EarlyStopping Callback
from tensorflow.keras.callbacks import EarlyStopping
earlystopping=EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)
earlystopping

## Meaning:-

### 1. **`EarlyStopping`**

* A **callback** in Keras that **stops training early** when a monitored metric stops improving.
* Prevents **overfitting** and **wasting time** on unnecessary epochs.

---

### 2. **`monitor='val_loss'`**

* This tells EarlyStopping **which metric to watch**.
* Here, it watches the **validation loss** (`val_loss`) after each epoch.
* If validation loss stops improving, it may stop training.

---

### 3. **`patience=5`**

* "How many epochs to wait for improvement before stopping."
* Example: If validation loss does not improve for **5 consecutive epochs**, training will stop.

---

### 4. **`restore_best_weights=True`**

* After stopping, it **restores the model weights from the epoch with the best validation loss**, not the last epoch.
* Ensures you end up with the **best model**, not the overfitted one.

---

✅ **In short:**
This code creates an EarlyStopping callback that will:

* Watch the validation loss,
* Stop training if it doesn’t improve for 5 epochs,
* Restore the model to the best version it saw during training.



In [45]:
## Training the model with earlystopping
model.fit(
    x_train,y_train,epochs=10,batch_size=32,
    validation_split=0.2,
    callbacks=[earlystopping]
)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 27s 40ms/step - accuracy: 0.5953 - loss: 58431.0273 - val_accuracy: 0.7900 - val_loss: 0.4557
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 24s 38ms/step - accuracy: 0.8469 - loss: 0.3574 - val_accuracy: 0.8574 - val_loss: 0.3436
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 24s 38ms/step - accuracy: 0.9126 - loss: 0.2230 - val_accuracy: 0.8592 - val_loss: 0.3422
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 24s 38ms/step - accuracy: 0.9483 - loss: 0.1394 - val_accuracy: 0.8662 - val_loss: 0.3709
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 24s 39ms/step - accuracy: 0.9632 - loss: 0.0981 - val_accuracy: 0.8558 - val_loss: 0.3994
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 24s 38ms/step - accuracy: 0.9446 - loss: 0.1655 - val_accuracy: 0.8364 - val_loss: 0.4220
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 24s 38ms/step - accuracy: 0.9762 - loss: 0.0737 - val_accuracy: 0.8386 - val_loss: 0.4758
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 24s 38ms/step - accuracy: 0.9862 - loss: 0.044

In [46]:
## save model file
model.save('Simple_rnn_imdb.h5')

In [47]:
## Mapping of word index to words (just for understanding)
word_index=imdb.get_word_index()
# word_index
reverse_word_index={value:key for key,value in word_index.items()}
reverse_word_index

{34701: 'fawn',
 52006: 'tsukino',
 52007: 'nunnery',
 16816: 'sonja',
 63951: 'vani',
 1408: 'woods',
 16115: 'spiders',
 2345: 'hanging',
 2289: 'woody',
 52008: 'trawling',
 52009: "hold's",
 11307: 'comically',
 40830: 'localized',
 30568: 'disobeying',
 52010: "'royale",
 40831: "harpo's",
 52011: 'canet',
 19313: 'aileen',
 52012: 'acurately',
 52013: "diplomat's",
 25242: 'rickman',
 6746: 'arranged',
 52014: 'rumbustious',
 52015: 'familiarness',
 52016: "spider'",
 68804: 'hahahah',
 52017: "wood'",
 40833: 'transvestism',
 34702: "hangin'",
 2338: 'bringing',
 40834: 'seamier',
 34703: 'wooded',
 52018: 'bravora',
 16817: 'grueling',
 1636: 'wooden',
 16818: 'wednesday',
 52019: "'prix",
 34704: 'altagracia',
 52020: 'circuitry',
 11585: 'crotch',
 57766: 'busybody',
 52021: "tart'n'tangy",
 14129: 'burgade',
 52023: 'thrace',
 11038: "tom's",
 52025: 'snuggles',
 29114: 'francesco',
 52027: 'complainers',
 52125: 'templarios',
 40835: '272',
 52028: '273',
 52130: 'zaniacs',

In [48]:
model.get_weights()

[array([[-0.02238492,  0.03014437,  0.00874239, ...,  0.0843261 ,
          0.07392152, -0.06086329],
        [-0.02294485,  0.04281711, -0.03872924, ...,  0.01039834,
          0.06216872, -0.01382526],
        [-0.01912131,  0.00018172,  0.01336936, ..., -0.00446653,
          0.03403391,  0.02489014],
        ...,
        [-0.0154385 ,  0.06164008, -0.03425594, ...,  0.01314288,
         -0.0258703 , -0.01250752],
        [ 0.03882989,  0.03660378,  0.01630597, ...,  0.06123463,
         -0.06837736, -0.00630289],
        [ 0.00084634, -0.16011804, -0.11484586, ...,  0.02711437,
          0.15392607,  0.06764618]], dtype=float32),
 array([[-0.05017781, -0.06606636,  0.12927942, ...,  0.05353887,
         -0.08372509, -0.10885364],
        [ 0.12757164, -0.04242342,  0.01723527, ...,  0.12768331,
         -0.03480041,  0.05287218],
        [ 0.14707425, -0.13216431,  0.06865022, ..., -0.05972406,
         -0.12841836, -0.09454652],
        ...,
        [-0.13252597, -0.105464  , -0.0

In [49]:
#Helper Functions
# Fxn to decode reviews
def decode_review(encoded_review):
    return ' '.join([reverse_word_index.get(i-3,"?")for i in encoded_review])
    """Keras IMDB shifts all real word indices by +3 to leave room for special tokens.
When decoding, you subtract 3 (i-3) to map back to the original word index"""
# function to preprocess user input
def preprocess_text(text):
    words = text.lower().split()
    encoded_review = [word_index.get(word,2)+3 for word in words]
    padded_review = sequence.pad_sequences([encoded_review],maxlen=500)
    return padded_review

## Explaination for encoded_review line:-

word_index is a dictionary mapping words → integer IDs (created when preparing the dataset).

word_index.get(word, 2) → looks up the index of each word:

If the word is found → return its index.

If not found → return 2 (reserved for “unknown word”).

+3 → shifts the index because:

0 is for padding,

1 is “start of sequence”,

2 is “unknown word”

So actual words start from index 3.

Example: If word_index["great"] = 45, then this becomes 45+3 = 48.


##  padded_review = sequence.pad_sequences([encoded_review], maxlen=500)

Neural networks expect inputs of the same length.

pad_sequences makes the review exactly length 500:

If shorter → adds 0s at the beginning.

If longer → truncates the extra words.

The [encoded_review] (inside a list) ensures it works with batch input shape.

Example: If review has 120 words, it will add 380 zeros in front.



In [50]:
## prediction fxn:-
def predict_sentiment(review):
    preprocessed_input = preprocess_text(review)
    prediction = model.predict(preprocessed_input)
    sentiment = "Postive" if prediction[0][0] > 0.5 else 'Negative'
    return sentiment,prediction[0][0]

In [51]:
## Prediction :-
sample_review = "The movie is fantastic,it is had thrilling action "

sentiment,score = predict_sentiment(sample_review)
print(f"Review :{sample_review}")
print(f"Sentiment : {sentiment}")
print(f"Prediction Score: {score}")



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 351ms/step
Review :The movie is fantastic,it is had thrilling action 
Sentiment : Postive
Prediction Score: 0.5766989588737488
